In [19]:
import configparser
import psycopg2
import time

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [34]:
def run_query(query):
    time_start = time.time()
    cur.execute(query)
    conn.commit()
    rows = cur.fetchall()
    print("Query:", end = " ")
    print(query)
    print("Result:", end = " ")
    for row in rows:
        print(row)
    print("Execution time:", time.time() - time_start, '\n')

In [37]:
print("Overview of Tables:")
run_query('select count(*) from staging_events')
run_query('select count(*) from staging_songs')
run_query('select count(*) from songplays')
run_query('select count(*) from songs')
run_query('select count(*) from users')
run_query('select count(*) from artists')
run_query('select count(*) from time')

Overview of Tables:
Query: select count(*) from staging_events
Result: (8056,)
Execution time: 0.8502557277679443 

Query: select count(*) from staging_songs
Result: (14896,)
Execution time: 0.9737410545349121 

Query: select count(*) from songplays
Result: (333,)
Execution time: 0.921616792678833 

Query: select count(*) from songs
Result: (14896,)
Execution time: 0.920811653137207 

Query: select count(*) from users
Result: (104,)
Execution time: 0.8538589477539062 

Query: select count(*) from artists
Result: (10025,)
Execution time: 0.9892339706420898 

Query: select count(*) from time
Result: (8023,)
Execution time: 0.9215090274810791 



(333,)
